# Social Network Model 
Djanira dos Santos Gomes, 2021

Python implementation of a Social Network Model created for my bachelor's thesis in Artificial Intelligence at Universiteit van Amsterdam. A detailed description of the model can be found in my thesis.

In [1]:
import numpy as np
import pandas as pd
from pandas.core.common import flatten
from itertools import combinations 

## Model initiation
Initiation of the agents, relations and positions dataframes. Relations are stored in a dataframe in the form of an adjacency matrix, where positive relations are reflexive and all relations are symmetric. Positions are stored in a dataframe where rows and columns represent agents and positions, respectively.  

In [2]:
# Define agents (A) 
A = ["a", "b", "c", "d"]

# Define the initial adjacency matrix dataframe containing initial relations 
# (friendship = 1, acquaintance = -1)
relations = [[1, -1, -1, 1],
             [-1, 1, -1, 0],
             [-1, -1, 1, -1],
             [1, 0, -1, 1]]
df_relations = pd.DataFrame(relations, columns=A, index=A)

# Define the initial dataframe containing positions of the agents on topics P={p}, Q={q}, R={r} 
positions = {"p":[0, 1, 1, 0], 
             "q":[1, 1, 1, 1], 
             "r":[1, 1, 0, 0]}
df_positions = pd.DataFrame(positions, index=A)

# Define nested dictionary to keep track of influencers
# Outer keys denote agents; inner key-value pairs show the kind of influencer,
# number of steps they have been influencers and their current positions
influencers = {"a":{"rig":0, "flex":0, "rig-steps":0, "flex-steps":0, "positions":[]},
               "b":{"rig":0, "flex":0, "rig-steps":0, "flex-steps":0, "positions":[]},
               "c":{"rig":0, "flex":0, "rig-steps":0, "flex-steps":0, "positions":[]},
               "d":{"rig":0, "flex":0, "rig-steps":0, "flex-steps":0, "positions":[]}}

In [3]:
df_relations

,a,b,c,d
a,1,-1,-1,1
b,-1,1,-1,0
c,-1,-1,1,-1
d,1,0,-1,1


In [4]:
df_positions

,p,q,r
a,0,1,1
b,1,1,1
c,1,1,0
d,0,1,0


## Additional functions needed for update rules

### $V_r$

In [5]:
"""
Returns list of agents that share 
position r (string) given agents (list)
""" 
def V_r(r, A):
    return df_positions.query("{} == 1".format(r)).index.tolist()

### Difference (with distance = len(difference))

In [6]:
"""
Returns difference (list of positions on which 
agents disagree) between two agents a, b (strings)
given positions (df)
"""
def diff(a, b, df_positions):
    
    diff_pos = []
    for position in df_positions.columns: 
        if df_positions.loc[a][position] != df_positions.loc[b][position]:
            diff_pos.append(position)
    return diff_pos

### Related agents 

In [7]:
"""
Returns all agents related to agent a 
(string) given relations (df)
"""
def related_agents(a, df_relations):
    return [column for column in df_relations.columns if df_relations.loc[a][column] != 0]

"""
Returns all agents with a positive relation 
with agent a (string) given relations (df) 
"""
def friends(a, df_relations):
    return [column for column in df_relations.columns if df_relations.loc[a][column] == 1]

"""
Returns all agents with a negative relation 
with agent a (string) given a relations dataframe 
"""
def acquaintances(a, df_relations):
    return [column for column in df_relations.columns if df_relations.loc[a][column] == -1]

## Social Influence update


In [8]:
"""
Performs social influence update given threshold (0<=t<=1), 
agents (list), relations (df) and positions (df)
and returns updated dataframe containing new positions 
"""
def soc_inf_update(t, A, df_relations, df_positions):
    # Initiate updated dataframe
    df_positions_upd = df_positions.copy()
    
    for agent in A: 
        related = related_agents(agent, df_relations)
        
        # Copy existing positions if agent has no related agents other than himself
        if related != [agent]:
            
            for position in df_positions.columns:
                # Adopt position if enough related agents support position
                agents_in_pos = set(related) & set(V_r(position, A))
                if (len(agents_in_pos) / len(related)) >= t:
                    df_positions_upd.loc[agent][position] = 1
                else:
                    df_positions_upd.loc[agent][position] = 0
                
    return df_positions_upd

In [9]:
df_next_positions = soc_inf_update(0.5, A, df_relations, df_positions)
df_next_positions

,p,q,r
a,1,1,1
b,1,1,1
c,1,1,1
d,0,1,0


## Friendship selection update
Note that the balance measure algorithm is written specifically for $\theta_1 = \frac{1}{3}$ and $\theta_2 = \frac{2}{3}$ (with t_fr as $\theta_1$ and t_ac as $\theta_2$).

In [10]:
# Set friendship and acquaintanceship thresholds
t_fr = 1/3
t_ac = 2/3
t_inf = 0.5

In [11]:
"""
Performs network update given thresholds (0<=t_fr,t_ac<=1), 
agents (list), relations (df) and positions (df)
and returns updated dataframe containing new relations
"""
def network_update(t_fr, t_ac, A, df_relations, df_positions):
    # Initiate updated dataframe
    df_relations_upd = df_relations.copy()
    
    positions_total = len(df_positions.columns)
    comb = combinations(A, 2)
    
    for i,j in list(comb):
        # Compute distance between i and j
        dist_rat = len(diff(i, j, df_positions)) / positions_total
        
        # Check if distance ratio between i,j permits friendship 
        if dist_rat <= t_fr:
                df_relations_upd.loc[i][j] = 1
                df_relations_upd.loc[j][i] = 1
        
        # Else, check if ratio permits acquaintanceship
        elif dist_rat <= t_ac:
                df_relations_upd.loc[i][j] = -1
                df_relations_upd.loc[j][i] = -1
        
        # Else, add no relation 
        else: 
            df_relations_upd.loc[i][j] = 0
            df_relations_upd.loc[j][i] = 0
        
    return df_relations_upd

In [12]:
df_next_relations = network_update(t_fr, t_ac, A, df_relations, df_positions)
df_next_relations

,a,b,c,d
a,1,1,-1,1
b,1,1,1,-1
c,-1,1,1,1
d,1,-1,1,1


## Influencers

### Additional functions

In [13]:
"""
Returns True if agent a (string) increases 
his proportion of friends within one update, 
returns False otherwise, given current and future 
relations (dataframes)
"""
def is_inf(a, df_relations, df_next_relations):
    # Define current and future friends and acquaintances
    fr = friends(a, df_relations)
    rel = related_agents(a, df_relations)
    fr_next = friends(a, df_next_relations)
    rel_next = related_agents(a, df_next_relations)

    # Check if upcoming ratio > current ratio (reflexive friendship relation prevents divisions by 0)
    ratio = len(fr) / len(rel)
    ratio_next = len(fr_next) / len(rel_next)
    if ratio_next > ratio:
        return True
    else:
        return False

In [14]:
"""
Returns True if agent a (string) maintains his 
position concerning r (string), returns False 
otherwise, given current and future 
positions (dataframes)
"""
def is_rigid_r(a, r, df_positions, df_next_positions):
    if df_positions.loc[a][r] == df_next_positions.loc[a][r]:
        return True
    else:
        return False 

### Rigid vs. flexible influencers 

In [15]:
"""
Returns True if agent a (string) is a 
(one-step) rigid influencer, returns False 
otherwise, given current and future 
relations and positions (dataframes)
"""
def is_rigid_inf(a, df_relations, df_positions, df_next_relations, df_next_positions):
    # Return False if any opinion of a has changed
    for position in df_positions.columns:
        if not is_rigid_r(a, position, df_positions, df_next_positions):
            return False
    
    # Return False if a is not an influencer
    if is_inf(a, df_relations, df_next_relations): 
        return True
    else: 
        return False 

In [16]:
"""
Returns True if agent a (string) is a 
(one-step) flexible influencer, returns False 
otherwise, given current and future 
relations and positions (dataframes)
"""
def is_flex_inf(a, df_relations, df_positions, df_next_relations, df_next_positions):
    # Return False if opinion has changed regarding every topic
    rigid_pos = []
    for position in df_positions.columns:
        if is_rigid_r(a, position, df_positions, df_next_positions):
            rigid_pos.append(position)
    if rigid_pos == []:
        return False
    
    # Return False if a is not an influencer
    if is_inf(a, df_relations, df_next_relations): 
        return True
    else: 
        return False 

### Update influencers

In [17]:
"""
Updates and returns the dictionary containing 
current influencers given agents (list) and 
current and future relations and positions (dataframes)
"""
def update_influencers(A, df_relations, df_positions, df_next_relations, df_next_positions, influencers):
    for agent in A: 
        if is_rigid_inf(agent, df_relations, df_positions, df_next_relations, df_next_positions):
            influencers[agent]["rig"] = 1
            influencers[agent]["flex"] = 1
            influencers[agent]["rig-steps"] += 1
            influencers[agent]["flex-steps"] += 1
            influencers[agent]["positions"] = df_positions.loc[agent].values.tolist()
        elif is_flex_inf(agent, df_relations, df_positions, df_next_relations, df_next_positions):
            influencers[agent]["rig"] = 0
            influencers[agent]["flex"] = 1
            influencers[agent]["rig-steps"] = 0
            influencers[agent]["flex-steps"] += 1
            influencers[agent]["positions"] = df_positions.loc[agent].values.tolist()
        else:
            influencers[agent]["rig"] = 0
            influencers[agent]["flex"] = 0
            influencers[agent]["rig-steps"] = 0
            influencers[agent]["flex-steps"] = 0
            influencers[agent]["positions"] = []
    return influencers
        

## Polarization measure: structural balance

### Additional functions

In [18]:
"""
Adds next layer to previous layers (nested list) 
extracted from a (component of a) social network 
model and removes those agents from the list of 
agents not included in any layers, given agents 
left in the model, and current relations and 
positions (dataframes)
"""
def add_next_layer(layers, agents_left, df_relations, df_positions):
    # Check if any agents are left for next layer 
    if agents_left == []:
        next_layer_exists = False
        return layers, next_layer_exists, agents_left
    
    current_layer = layers[-1]
    next_layer = []
    
    # Get neighbours of agents in current layer
    for i in current_layer:
        rel_i = related_agents(i, df_relations)
        
        # Add agents to new layer if not present in previous layers / next layer
        for j in rel_i:
            
            if j in agents_left:
                next_layer.append(j)
                agents_left.remove(j)
    
    # Add next layer to previous layers if not empty
    if next_layer != []:
        layers.append(next_layer)
        next_layer_exists = True
    else: 
        next_layer_exists = False

    return layers, next_layer_exists, agents_left

In [19]:
"""
Extracts layers of neighbours starting 
from a root agent A[0] in a network and 
establishes if the network is connected, 
given agents (list) and relations and 
positions (dataframes)
"""
def construct_layers(A, df_relations, df_positions):
    # Set the root to agent A[0]
    agents_left = A[1:]
    layers = [[A[0]]]
    next_layer_exists = True
    is_connected = False
    
    # Construct layers from the part of the network connected to the root (A[0])
    while next_layer_exists == True:
        layers, next_layer_exists, agents_left = add_next_layer(layers, agents_left, 
                                                                df_relations, df_positions)
    # If no agents left, network is connected    
    if agents_left == []:
        is_connected = True
    
    return layers, is_connected

### Balanced Division

In [20]:
"""
Returns two lists of agents X and Y if 
a balanced division exists and two empty 
lists otherwise, given agents (list), and 
relations and positions (dataframes) 
"""
def balanced_division(A, df_relations, df_positions):
    # Construct layers
    layers, is_connected = construct_layers(A, df_relations, df_positions)
    
    # If connected, try placing agents into groups per layer
    if is_connected:
        X = [A[0]]
        Y = []
        for i in range(1, len(layers)):
            # Get all agents in layers up to current layer
            prev_layers_list = list(flatten(layers[:i+1]))
            
            for agent in layers[i]:
                # Determine friends and acquaintances in previous layers
                fr = friends(agent, df_relations)
                ac = acquaintances(agent, df_relations)     
                friends_prev_layers = set(fr) & set(prev_layers_list)
                ac_prev_layers = set(ac) & set(prev_layers_list)
            
                # Return False if a friend and acquaintance share a group
                if any(x in X for x 
                       in friends_prev_layers) and any(x in X for x in ac_prev_layers):
                        return [], []
                elif any(x in Y for x 
                         in friends_prev_layers) and any(x in Y for x in ac_prev_layers):
                            return [], []
                
                # Else place in a group dependent on related agents
                elif any(x in X for x 
                         in friends_prev_layers) or any(x in Y for x in ac_prev_layers):
                            X.append(agent)
                elif any(x in Y for x 
                         in friends_prev_layers) or any(x in X for x in ac_prev_layers):
                            Y.append(agent)
                        
        return X, Y
    
    # If disconnected, connected components form balanced division
    else:
        X = list(flatten(layers))
        Y = list(set(A).difference(set(X)))
        return X, Y

In [21]:
balanced_division(A, df_relations, df_positions)

([], [])

In [22]:
"""
Computes current influencers; performs both 
updates alternately for a given number of steps 
(int), given influence threshold (0<=t<=1) and 
relationship thresholds (t_fr = 1/3, t_ac = 2/3),
agents (list), and relations and positions 
(dataframes); returns balanced division (otherwise 
two empty lists), influencers, and updated model
"""
def run_model(steps, t_inf, t_fr, t_ac, A, df_relations, df_positions, influencers):
    display(df_relations)
    display(df_positions)
        
    for i in range(steps): 
        influencers = update_influencers(A, df_relations, df_positions, df_next_relations, 
                                         df_next_positions, influencers)
        
        df_positions = soc_inf_update(t_inf, A, df_relations, df_positions)
        df_relations = network_update(t_fr, t_ac, A, df_relations, df_positions)
        
        display(df_relations)
        display(df_positions)
        
        X, Y = balanced_division(A, df_relations, df_positions)
        print("Balanced division:", X, Y)
        
    return X, Y, influencers, df_relations, df_positions 

In [23]:
X, Y, influencers, df_relations_3, df_positions_3 = run_model(2, t_inf, t_fr, t_ac, A, 
                                           df_relations, df_positions, influencers)     

,a,b,c,d
a,1,-1,-1,1
b,-1,1,-1,0
c,-1,-1,1,-1
d,1,0,-1,1


,p,q,r
a,0,1,1
b,1,1,1
c,1,1,0
d,0,1,0


,a,b,c,d
a,1,1,1,-1
b,1,1,1,-1
c,1,1,1,-1
d,-1,-1,-1,1


,p,q,r
a,1,1,1
b,1,1,1
c,1,1,1
d,0,1,0


Balanced division: ['a', 'b', 'c'] ['d']


,a,b,c,d
a,1,1,1,1
b,1,1,1,1
c,1,1,1,1
d,1,1,1,1


,p,q,r
a,1,1,1
b,1,1,1
c,1,1,1
d,1,1,1


Balanced division: ['a', 'b', 'c', 'd'] []
